In [1]:
import os

In [2]:
%pwd

'f:\\OneDrive - MSFT\\wine_quality_ML\\Machine-Learning-End-to-End-Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'f:\\OneDrive - MSFT\\wine_quality_ML\\Machine-Learning-End-to-End-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    all_schema: dict
    unzip_data_dir: Path


In [6]:
# configuration Manager
from src.MLProject_WineQT.constants.const import *
from src.MLProject_WineQT.utils.common import read_yaml, create_directories
import pandas as pd

In [7]:
import pandas as pd
from src.MLProject_WineQT.constants.const import *
from src.MLProject_WineQT.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)  # Ensure this loads the YAML correctly
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        # Create directories specified in the config
        create_directories([self.config['artifacts_root']])  # Adjusted to use key-based access
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config['data_validation']  # Correct key access method
        schema = self.schema['COLUMNS']  # Correct access to schema
        
        # Create the data validation directory if it doesn't exist
        create_directories([config['root_dir']])
        
        return DataValidationConfig(
            root_dir=Path(config['root_dir']),
            STATUS_FILE=config['STATUS_FILE'],
            unzip_data_dir=Path(config['unzip_data_dir']),
            all_schema=schema
        )


In [8]:
import os

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
    
    def validation_all_columns(self) -> bool:
        try:
            validation_status = True  # Default to True
            
            # Check if the file exists
            if not os.path.exists(self.config.unzip_data_dir):
                raise FileNotFoundError(f"File {self.config.unzip_data_dir} does not exist.")
            
            # Read the CSV file from the specified directory
            data = pd.read_csv(self.config.unzip_data_dir)  # Ensure this is the full file path
            all_cols = list(data.columns)
            
            # Schema keys
            all_schema = self.config.all_schema.keys()
            
            # Validate each column
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation failed for column: {col}\n")
                else:
                    with open(self.config.STATUS_FILE, 'a') as f:  # Append mode
                        f.write(f"Validation passed for column: {col}\n")
            
            return validation_status
        except Exception as e:
            raise e


In [9]:
try:
    config = ConfigurationManager()  # Initialize configuration manager
    data_validation_config = config.get_data_validation_config()  # Fetch data validation config
    data_validation = DataValidation(config=data_validation_config)  # Create DataValidation instance
    data_validation.validation_all_columns()  # Perform validation
except Exception as e:
    raise e


[2024-10-04 14:42:24,320: INFO: common: yaml file:config\config.yaml load successfuly]
[2024-10-04 14:42:24,325: INFO: common: yaml file:params.yaml load successfuly]
[2024-10-04 14:42:24,333: INFO: common: yaml file:schema.yaml load successfuly]
[2024-10-04 14:42:24,337: INFO: common: created directories at: artifacts]
[2024-10-04 14:42:24,377: INFO: common: created directories at: artifacts/data_validation]


FileNotFoundError: File artifacts\data_ingestion\unzipped_data\data.csv does not exist.